## I CALL IT VERSION ONE

### Creating a script that does all:
#### 1. Posts Extraction from instagram profile of past months
#### 2. Conversion of it's speech audio into text (Transcription)
#### 3. Extraction of the Matrix <u>with or without instagram API</u>
#### 4. matrix we needed:
#### &ensp; &ensp; &ensp; shares, saves, reach, likes, comments, description, audio used, date & time.

#### &ensp;&ensp;&ensp;# &ensp; For implementing this things use any bootstrap method, EIther Exploitation or legal method.

### Damn Installation

In [1]:
!pip install instaloader
!pip install requests
!pip install pyaudio
!pip install SpeechRecognition
!pip install pydub
!pip install tkinter
!pip install noisereduce
!pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for instaloader: filename=instaloader-4.13-py3-none-any.whl size=67481 sha256=0b795665e047adfa601fdcf861a7b3e7a3e17cdd072812a627af7cbf45b0315e
  Stored in directory: /root/.cache/pip/wheels/2e/9b/eb/43b45fb5dadc042e7512c74e2bedbb5c740efb37fb9023d23d
Successfully built instaloader
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: ERROR: Failed to build installable wheels f

### upgraded version of above code - I think this is it, the final code.

In [ ]:
#for single link
import instaloader
from instaloader.exceptions import ConnectionException, InstaloaderException
import re
import os
import requests
from moviepy.editor import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip
###########################################################################################
import speech_recognition as sr
from pydub import AudioSegment
import time

class InstagramPostExtractor:
    def __init__(self):
        # Initialize Instaloader
        self.loader = instaloader.Instaloader()
        self.login()

    def login(self):
        # Retrieve credentials from environment variables
        username = "Avii_analytics_labs"
        password = "Avii@29,"

        # username = "okaeriweebs"
        # password = "makeithappen313"

        if username and password:
            try:
                self.loader.login(username, password)
                print("Logged in successfully.")
            except ConnectionException as e:
                print(f"Connection error: {e}")
            except InstaloaderException as e:
                print(f"Instaloader exception: {e}")
            except Exception as e:
                print(f"An error occurred during login: {e}")
        else:
            print("Instagram credentials are not set in environment variables.")
    url = input("Enter the Instagram post URL: ")
    shortcode = re.search(r'/p/([^/]+)/', url).group(1)
    audio_path = os.path.join('downloads', f'{shortcode}.mp3')
    print(audio_path)
    def extract_post_info(self, post_url):
        post_url = post_url.strip()

        # Validate URL format
        if not re.match(r'^https://www\.instagram\.com/p/[^/]+/$', post_url):
            print("Invalid URL format. Please enter a valid Instagram post URL.")
            return

        # Extract the post shortcode from the URL
        try:
            shortcode = re.search(r'/p/([^/]+)/', post_url).group(1)
        except AttributeError:
            print("Unable to extract shortcode from URL.")
            return

        try:
            # Load the post by shortcode
            post = instaloader.Post.from_shortcode(self.loader.context, shortcode)

            # Extract post information
            post_info = {
                'Shortcode': post.shortcode,
                'Caption': post.caption,
                'Likes': post.likes,
                'Comments': post.comments,
                'Date': post.date_utc,
                'Is Video': post.is_video,
                'Video URL': post.video_url if post.is_video else "N/A",
                # Placeholders for metrics not available via Instaloader
                'Saved': "N/A",
                'Reach': "N/A",
                'Shared': "N/A",
                'Music': "N/A"  # Not available
            }

            # Display post information
            for key, value in post_info.items():
                print(f"{key}: {value}")

            # Download the video if it exists
            if post.is_video:
                self.download_video(post.video_url, post.shortcode)

        except InstaloaderException as e:
            print(f"Instaloader exception: {e}")
        except ConnectionException:
            print("Connection error occurred.")
        except Exception as e:
            print(f"An error occurred: {e}")

    def download_video(self, video_url, shortcode):
        # Create a directory for downloads
        if not os.path.exists('downloads'):
            os.makedirs('downloads')

        # Download the video
        video_response = requests.get(video_url, stream=True)
        video_path = os.path.join('downloads', f'{shortcode}.mp4')

        with open(video_path, 'wb') as video_file:
            for chunk in video_response.iter_content(chunk_size=8192):
                if chunk:
                    video_file.write(chunk)

        print(f"Video downloaded to {video_path}")

        # Convert video to audio
        self.convert_video_to_audio(video_path, shortcode)

    def convert_video_to_audio(self, video_path, shortcode):
        # Load the video file
        video_clip = VideoFileClip(video_path)
        audio_clip = video_clip.audio

        # Save audio as mp3
        audio_path = os.path.join('downloads', f'{shortcode}.mp3')

        audio_clip.write_audiofile(audio_path)
        audio_clip.close()
        video_clip.close()

        print(f"Audio converted and saved to {audio_path}")

# Main script
if __name__ == "__main__":
    # times = int(input("Enter num of posts you have to check : "))
    extractor = InstagramPostExtractor()
    # url = input("Enter the Instagram post URL: ")
    extractor.extract_post_info(extractor.url)

time.sleep(10)
####################################################################################################################
def convert_mp3_to_wav(mp3_path, wav_path):
    """Convert MP3 audio file to WAV format."""
    audio = AudioSegment.from_mp3(mp3_path)
    audio.export(wav_path, format='wav')
    print(f"Converted {mp3_path} to {wav_path}")

def transcribe_audio_chunks(audio_path, chunk_length_ms=30000):
    """Transcribe audio file to text in chunks."""
    recognizer = sr.Recognizer()

    # Convert audio to WAV if necessary
    wav_path = audio_path.replace('.mp3', '.wav')
    if audio_path.endswith('.mp3'):
        convert_mp3_to_wav(audio_path, wav_path)
    else:
        wav_path = audio_path

    audio = AudioSegment.from_wav(wav_path)
    audio_length = len(audio)
    transcript = ""

    print(f"Transcribing {wav_path} in chunks of {chunk_length_ms / 1000} seconds...")

    for start in range(0, audio_length, chunk_length_ms):
        end = min(start + chunk_length_ms, audio_length)
        chunk = audio[start:end]
        chunk_path = f"{wav_path}_chunk_{start // chunk_length_ms}.wav"
        chunk.export(chunk_path, format='wav')

        try:
            with sr.AudioFile(chunk_path) as source:
                audio_data = recognizer.record(source)
                chunk_text = recognizer.recognize_google(audio_data)
                transcript += chunk_text + " "
                print(f"Chunk {start // chunk_length_ms} transcribed.")

        except sr.UnknownValueError:
            print(f"Chunk {start // chunk_length_ms} could not be understood.")
        except sr.RequestError as e:
            print(f"Could not request results for chunk {start // chunk_length_ms}; {e}")

        # Clean up temporary chunk file
        os.remove(chunk_path)
    # Save the full transcript
    transcript_path = os.path.join(os.path.dirname(wav_path), os.path.basename(wav_path).replace('.wav', '.txt'))
    with open(transcript_path, 'w') as file:
        file.write(transcript)
    print(f"Transcription saved to {transcript_path}")
############
# Main script
if __name__ == "__main__":
    # audio_path = input("Enter the path to the audio file (MP3 or WAV): ")
    audio_pth = extractor.audio_path
    if not os.path.isfile(audio_pth):
        print("The provided file path does not exist.")
    else:
        transcribe_audio_chunks(audio_pth)


Enter the Instagram post URL: https://www.instagram.com/p/C-TIiK4y7_w/
downloads/C-TIiK4y7_w.mp3
Logged in successfully.
Shortcode: C-TIiK4y7_w
Caption: Discussing my Ayhuasca experience with Elon Musk, part of an 8.5 hour podcast up on YouTube now. I'm still working on a video about the time I spent in the jungle with my friend, the amazing @paulrosolie. It was an experience I'll never forget 👊
Likes: 27635
Comments: 674
Date: 2024-08-05 18:54:18
Is Video: True
Video URL: https://scontent-iad3-2.cdninstagram.com/o1/v/t16/f2/m69/An_2IZFZqzYh3oTomiGXU-KBwadksZQ27PNReRhFl8ZEuwsSKQpogp1Othyg6-bKXfBqtFoIjtEpoFy8S1yNlNH3.mp4?efg=eyJxZV9ncm91cHMiOiJbXCJpZ193ZWJfZGVsaXZlcnlfdnRzX290ZlwiXSIsInZlbmNvZGVfdGFnIjoidnRzX3ZvZF91cmxnZW4uY2xpcHMuYzIuMTA4MC5iYXNlbGluZSJ9&_nc_ht=scontent-iad3-2.cdninstagram.com&_nc_cat=106&vs=1020832826350527_4195606892&_nc_vs=HBksFQIYOnBhc3N0aHJvdWdoX2V2ZXJzdG9yZS9HSUNXbUFBMy1yX002NUlDQUN5OUR1ZmpEU0ZnYnBSMUFBQUYVAALIAQAVAhg6cGFzc3Rocm91Z2hfZXZlcnN0b3JlL0dLNDJGUnRKNllJd

MoviePy - Done.
Audio converted and saved to downloads/C-TIiK4y7_w.mp3
Converted downloads/C-TIiK4y7_w.mp3 to downloads/C-TIiK4y7_w.wav
Transcribing downloads/C-TIiK4y7_w.wav in chunks of 30.0 seconds...
Chunk 0 transcribed.
Chunk 1 could not be understood.
Chunk 2 transcribed.
Chunk 3 transcribed.
Chunk 4 transcribed.
Chunk 5 transcribed.
Chunk 6 transcribed.
Chunk 7 transcribed.
Chunk 8 transcribed.
Chunk 9 could not be understood.
Chunk 10 transcribed.
Chunk 11 transcribed.
Transcription saved to downloads/C-TIiK4y7_w.txt


### doing something as a trial - for taking n number of links

In [ ]:
import instaloader
from instaloader.exceptions import ConnectionException, InstaloaderException

class InstagramVideoLinkExtractor:
    def __init__(self, username, password):
        self.loader = instaloader.Instaloader()
        self.login(username, password)

    def login(self, username, password):
        try:
            self.loader.login(username, password)
            print("Logged in successfully.")
        except ConnectionException as e:
            print(f"Connection error: {e}")
        except InstaloaderException as e:
            print(f"Instaloader exception: {e}")
        except Exception as e:
            print(f"An error occurred during login: {e}")

    def extract_video_links(self, target_username, n):
        try:
            profile = instaloader.Profile.from_username(self.loader.context, target_username)
            video_links = []

            print(f"Extracting up to {n} video post links from @{target_username}...")

            for post in profile.get_posts():
                if post.is_video:
                    video_links.append(f"https://www.instagram.com/p/{post.shortcode}/")
                if len(video_links) >= n:
                    break

            return video_links
        except InstaloaderException as e:
            print(f"An error occurred: {e}")
            return []

# Main script
if __name__ == "__main__":
    # Instagram login credentials
    your_username = "avii_analytics_labs"
    your_password = "Avii@29,"

    # Instagram profile to extract video links from
    target_profile = input("Enter the Instagram username to extract video links from: ")

    # Number of video links to extract
    num_videos = int(input("Enter the number of video links to extract: "))

    extractor = InstagramVideoLinkExtractor(your_username, your_password)
    video_links = extractor.extract_video_links(target_profile, num_videos)

    if video_links:
        print(f"Found {len(video_links)} video links:")
        for link in video_links:
            print(link)
    else:
        print("No video links found or an error occurred.")
#######################################################################################################################################
def save_links_to_file(links, filename):
    """
    Save a list of links to a text file, each on a separate line.

    :param links: List of video post links.
    :param filename: The name of the text file to save the links to.
    """
    try:
        with open(filename, 'w') as file:
            for link in links:
                file.write(f"{link}\n")
        print(f"Links saved to {filename}")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")


# Main script
if __name__ == "__main__":
    # Example list of links (replace this with the actual list from the extractor)
    video_links = [
        "https://www.instagram.com/p/EXAMPLE1/",
        "https://www.instagram.com/p/EXAMPLE2/",
        "https://www.instagram.com/p/EXAMPLE3/"
    ]

    # Name of the output text file
    output_filename = "video_links.txt"

    # Save the links to the file
    save_links_to_file(video_links, output_filename)


Enter the Instagram username to extract video links from: khalidwalb
Enter the number of video links to extract: 10
Logged in successfully.
Extracting up to 10 video post links from @khalidwalb...
Found 10 video links:
https://www.instagram.com/p/C-mrCpXtAyL/
https://www.instagram.com/p/C-e7FLtgHTC/
https://www.instagram.com/p/C-UnxYqxb7J/
https://www.instagram.com/p/C-Hv3mosMsm/
https://www.instagram.com/p/C-CnQMCyPZz/
https://www.instagram.com/p/C962lkMsoWO/
https://www.instagram.com/p/C9rZjN-h-zY/
https://www.instagram.com/p/C9mSaoxSO60/
https://www.instagram.com/p/C9ejpW_xmBg/
https://www.instagram.com/p/C9UQp1ivmZn/
Links saved to video_links.txt


### This is Hellllllll.....................

In [ ]:
import instaloader
from instaloader.exceptions import ConnectionException, InstaloaderException
import re
import os
import requests
from moviepy.editor import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip
import speech_recognition as sr
from pydub import AudioSegment
import time

class InstagramPostExtractor:
    def __init__(self):
        # Initialize Instaloader
        self.loader = instaloader.Instaloader()
        self.login()

    def login(self):
        # Retrieve credentials from environment variables
        username = "Avii_analytics_labs"
        password = "Avii@29,"

        if username and password:
            try:
                self.loader.login(username, password)
                print("Logged in successfully.")
            except ConnectionException as e:
                print(f"Connection error: {e}")
            except InstaloaderException as e:
                print(f"Instaloader exception: {e}")
            except Exception as e:
                print(f"An error occurred during login: {e}")
        else:
            print("Instagram credentials are not set in environment variables.")

    def extract_post_info(self, post_url):
        post_url = post_url.strip()

        # Validate URL format
        if not re.match(r'^https://www\.instagram\.com/p/[^/]+/$', post_url):
            print("Invalid URL format. Please enter a valid Instagram post URL.")
            return

        # Extract the post shortcode from the URL
        try:
            shortcode = re.search(r'/p/([^/]+)/', post_url).group(1)
        except AttributeError:
            print("Unable to extract shortcode from URL.")
            return

        try:
            # Load the post by shortcode
            post = instaloader.Post.from_shortcode(self.loader.context, shortcode)

            # Extract post information
            post_info = {
                'Shortcode': post.shortcode,
                'Caption': post.caption,
                'Likes': post.likes,
                'Comments': post.comments,
                'Date': post.date_utc,
                'Is Video': post.is_video,
                'Video URL': post.video_url if post.is_video else "N/A",
                # Placeholders for metrics not available via Instaloader
                'Saved': "N/A",
                'Reach': "N/A",
                'Shared': "N/A",
                'Music': "N/A"  # Not available
            }

            # Display post information
            for key, value in post_info.items():
                print(f"{key}: {value}")

            # Download the video if it exists
            if post.is_video:
                self.download_video(post.video_url, post.shortcode)

        except InstaloaderException as e:
            print(f"Instaloader exception: {e}")
        except ConnectionException:
            print("Connection error occurred.")
        except Exception as e:
            print(f"An error occurred: {e}")

    def download_video(self, video_url, shortcode):
        # Create a directory for downloads
        if not os.path.exists('downloads'):
            os.makedirs('downloads')

        # Download the video
        video_response = requests.get(video_url, stream=True)
        video_path = os.path.join('downloads', f'{shortcode}.mp4')

        with open(video_path, 'wb') as video_file:
            for chunk in video_response.iter_content(chunk_size=8192):
                if chunk:
                    video_file.write(chunk)

        print(f"Video downloaded to {video_path}")

        # Check if the video file was actually downloaded
        if os.path.exists(video_path):
            # Convert video to audio
            return self.convert_video_to_audio(video_path, shortcode)
        else:
            print(f"Error: Video download failed for {video_url}")
            return None  # Indicate failure

    def convert_video_to_audio(self, video_path, shortcode):
        # Load the video file
        video_clip = VideoFileClip(video_path)
        audio_clip = video_clip.audio

        # Save audio as mp3
        audio_path = os.path.join('downloads', f'{shortcode}.mp3')
        audio_clip.write_audiofile(audio_path)
        audio_clip.close()
        video_clip.close()

        print(f"Audio converted and saved to {audio_path}")
        return audio_path

class InstagramPostProcessor(InstagramPostExtractor):
    def __init__(self):
        super().__init__()

    def process_links_from_file(self, file_path, extractor): # Add 'self' as the first argument
        """Process multiple Instagram post URLs from a file."""
        try:
            with open(file_path, 'r') as file:
                urls = file.readlines()
                for url in urls:
                    extractor.extract_post_info(url)
                    # Extract shortcode from the URL
                    shortcode = re.search(r'/p/([^/]+)/', url).group(1)
                    audio_path = extractor.convert_video_to_audio('downloads/' + shortcode + '.mp4', shortcode)
                    if audio_path: # Check if audio_path is not None (indicating successful conversion)
                        if os.path.isfile(audio_path):
                            self.transcribe_audio_chunks(audio_path)
                        time.sleep(10)
                    else:
                        print(f"Skipping transcription for {shortcode} due to missing audio.")

        except FileNotFoundError:
            print(f"File {file_path} not found.")
        except AttributeError:
            print(f"Unable to extract shortcode from URL: {url}") # Handle cases where shortcode extraction fails

    def transcribe_audio_chunks(self, audio_path, chunk_length_ms=30000):
        """Transcribe audio file to text in chunks."""
        recognizer = sr.Recognizer()

        # Convert audio to WAV if necessary
        wav_path = audio_path.replace('.mp3', '.wav')
        if audio_path.endswith('.mp3'):
            self.convert_mp3_to_wav(audio_path, wav_path)
        else:
            wav_path = audio_path

        audio = AudioSegment.from_wav(wav_path)
        audio_length = len(audio)
        transcript = ""

        print(f"Transcribing {wav_path} in chunks of {chunk_length_ms / 1000} seconds...")

        for start in range(0, audio_length, chunk_length_ms):
            end = min(start + chunk_length_ms, audio_length)
            chunk = audio[start:end]
            chunk_path = f"{wav_path}_chunk_{start // chunk_length_ms}.wav"
            chunk.export(chunk_path, format='wav')

            try:
                with sr.AudioFile(chunk_path) as source:
                    audio_data = recognizer.record(source)
                    chunk_text = recognizer.recognize_google(audio_data)
                    transcript += chunk_text + " "
                    print(f"Chunk {start // chunk_length_ms} transcribed.")

            except sr.UnknownValueError:
                print(f"Chunk {start // chunk_length_ms} could not be understood.")
            except sr.RequestError as e:
                print(f"Could not request results for chunk {start // chunk_length_ms}; {e}")

            # Clean up temporary chunk file
            os.remove(chunk_path)
        # Save the full transcript
        transcript_path = os.path.join(os.path.dirname(wav_path), os.path.basename(wav_path).replace('.wav', '.txt'))
        with open(transcript_path, 'w') as file:
            file.write(transcript)
        print(f"Transcription saved to {transcript_path}")

    def convert_mp3_to_wav(self, mp3_path, wav_path):
        """Convert MP3 audio file to WAV format."""
        audio = AudioSegment.from_mp3(mp3_path)
        audio.export(wav_path, format='wav')
        print(f"Converted {mp3_path} to {wav_path}")

# Main script
if __name__ == "__main__":
    processor = InstagramPostProcessor()
    extractor = InstagramPostExtractor()
    file_path = input("Enter the path to the file containing Instagram post URLs: ")
    processor.process_links_from_file(file_path, extractor)


### Some trial

In [17]:
import requests
from bs4 import BeautifulSoup

def get_instagram_user_id(username):
    url = f"https://www.instagram.com/{username}/"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        scripts = soup.find_all('script')
        for script in scripts:
            if 'profilePage_' in script.text:
                user_id = script.text.split('profilePage_')[1].split('"')[0]
                return user_id
    return None

username = "avii_analytics_labs"
user_id = get_instagram_user_id(username)
print(f"User ID for {username}: {user_id}")


User ID for avii_analytics_labs: 62168458171


In [27]:
import requests

# Replace with your access token
ACCESS_TOKEN = "EAAFo991WrroBO3gG5ZAdXD6Hozby9Ka9oytSNXRaPO1l8U9ZCt8EVB3JzS1h3t7iYuO5vvgfuomXZC9WymVc4M55guZBI4wfe0kfrh3EQnNy1TA5NIkBUXpUWqnwMJvdSyZCFkjnfE6AZAYve4oXtKYUQ0wZCM5MloUldwgP1jTpWZBxXruUa3RJZBIOrNVMAwH0DXeJx1UMudv6zRB5ZCIoUxV5oxnAZDZD"

def get_instagram_user_id(access_token):
    url = f"https://graph.instagram.com/me?fields=id,username&access_token={access_token}"
    response = requests.get(url)
    if response.status_code == 200:
        user_info = response.json()
        return user_info['id'], user_info['username']
    else:
        print(f"Error getting user ID: {response.status_code}, {response.text}")
        return None, None

def get_instagram_media_ids(user_id, access_token):
    url = f"https://graph.instagram.com/{user_id}/media?access_token={access_token}"
    response = requests.get(url)
    if response.status_code == 200:
        media_data = response.json()
        media_ids = [media['id'] for media in media_data['data']]
        return media_ids
    else:
        print(f"Error getting media IDs: {response.status_code}, {response.text}")
        return []

def get_instagram_post_details(media_id, access_token):
    url = f"https://graph.instagram.com/{media_id}?fields=id,caption,media_type,media_url,permalink,thumbnail_url,timestamp,username&access_token={access_token}"
    response = requests.get(url)
    if response.status_code == 200:
        post_info = response.json()
        return post_info
    else:
        print(f"Error getting post details: {response.status_code}, {response.text}")
        return None

if __name__ == "__main__":
    # Step 1: Get Instagram User ID
    user_id, username = get_instagram_user_id(ACCESS_TOKEN)
    if user_id:
        print(f"User ID: {user_id}")
        print(f"Username: {username}")

        # Step 2: Get Media (Post) IDs
        media_ids = get_instagram_media_ids(user_id, ACCESS_TOKEN)
        if media_ids:
            print(f"Found {len(media_ids)} media items.")

            # Step 3: Get Details for Each Post
            for media_id in media_ids:
                post_details = get_instagram_post_details(media_id, ACCESS_TOKEN)
                if post_details:
                    print("\nPost Details:")
                    for key, value in post_details.items():
                        print(f"{key}: {value}")


Error getting user ID: 400, Sorry, this content isn't available right now


In [23]:
import requests

access_token = "EAAFo991WrroBO14hzcg9rZBNxZCcl8sqOVZANrqeCBNDchXXMVfpx6vyNVWisZBpOfpKPkn0jvXDoDB9L344xTRqDOgn87Y2TYcW0SQo6YCZAzumKhCekgGmUVF5lhTS3djQ6Q0dnqcCy4OyIIwmW9mLyfMgBaFMWZCSx6LUOozbdHmpgxCJOmxJGEpVPxLDcCHrXR8W8u0xe91isgzSLuuXgSTDM608XlaQDkaDbzrWXtDvG84BW7CHVz1obJFrs4v4d96gZDZD"
ig_user_id = "62168458171"
post_id = "3776603275"

url = f"https://graph.facebook.com/v14.0/{post_id}/insights"
params = {
    'metric': 'engagement,impressions,reach,saved',
    'access_token': access_token
}

response = requests.get(url, params=params)
data = response.json()

if 'data' in data:
    for insight in data['data']:
        print(f"{insight['title']}: {insight['values'][0]['value']}")
else:
    print(f"Error: {data.get('error', 'Unknown error')}")


Error: {'message': "Unsupported get request. Object with ID '3776603275' does not exist, cannot be loaded due to missing permissions, or does not support this operation. Please read the Graph API documentation at https://developers.facebook.com/docs/graph-api", 'type': 'GraphMethodException', 'code': 100, 'error_subcode': 33, 'fbtrace_id': 'AQ1MQvI2Tv3xaQlUHzZ0A3T'}


In [ ]:
import requests
import json

class InstagramAPI:
    def __init__(self, access_token, instagram_account_id):
        self.access_token = access_token
        self.instagram_account_id = instagram_account_id
        self.base_url = "https://graph.facebook.com/v17.0"

    def get_user_media(self):
        url = f"{self.base_url}/{self.instagram_account_id}/media"
        params = {
            'fields': 'id,caption,media_type,media_url,thumbnail_url,permalink',
            'access_token': self.access_token
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()['data']
        else:
            print(f"Error: {response.status_code}")
            print(response.text)
            return []

    def get_post_metrics(self, media_id):
        url = f"{self.base_url}/{media_id}/insights"
        params = {
            'metric': 'engagement,impressions,reach,saved,shares',
            'access_token': self.access_token
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()['data']
        else:
            print(f"Error: {response.status_code}")
            print(response.text)
            return {}

    def extract_post_info(self, post_url):
        media_data = self.get_user_media()

        for media in media_data:
            if media['permalink'] == post_url:
                media_id = media['id']
                metrics = self.get_post_metrics(media_id)

                post_info = {
                    'ID': media['id'],
                    'Caption': media.get('caption', 'N/A'),
                    'Media Type': media['media_type'],
                    'Media URL': media.get('media_url', 'N/A'),
                    'Thumbnail URL': media.get('thumbnail_url', 'N/A'),
                    'Permalink': media['permalink'],
                    'Saved': metrics.get('saved', {'values': [{'value': 'N/A'}]}),
                    'Reach': metrics.get('reach', {'values': [{'value': 'N/A'}]}),
                    'Shared': metrics.get('shares', {'values': [{'value': 'N/A'}]}),
                    'Music': "N/A"  # Placeholder as music info isn't available via API
                }

                print(json.dumps(post_info, indent=4))
                return

        print("Post not found or invalid URL.")

# Main script
if __name__ == "__main__":
    ACCESS_TOKEN = 'YOUR_ACCESS_TOKEN'
    INSTAGRAM_ACCOUNT_ID = 'YOUR_INSTAGRAM_ACCOUNT_ID'

    instagram_api = InstagramAPI(ACCESS_TOKEN, INSTAGRAM_ACCOUNT_ID)
    post_url = input("Enter the Instagram post URL: ")
    instagram_api.extract_post_info(post_url)


Enter the Instagram post URL: https://www.instagram.com/p/C-mrCpXtAyL/
Error: 400
{"error":{"message":"Got unexpected null","type":"OAuthException","code":190,"fbtrace_id":"ATktooz4gVgPXiPj6fntIQI"}}
Post not found or invalid URL.


### New Thing

In [12]:
import instaloader
import requests
from bs4 import BeautifulSoup
import json

class InstagramPostExtractor:
    def __init__(self):
        # Initialize Instaloader
        self.loader = instaloader.Instaloader()
        self.login()

    def login(self):
        # Your Instagram credentials
        username = 'avii_analytics_labs'
        password = 'Avii@29,'

        try:
            self.loader.login(username, password)
            print("Logged in successfully.")
        except Exception as e:
            print(f"Login failed: {e}")

    def extract_post_info(self, post_url):
        post_url = post_url.strip()

        # Extract the post shortcode from the URL
        shortcode = post_url.split("/")[-2]
        print(f"Shortcode: {shortcode}")

        # Fetching the post data using Instaloader
        # try:
        post = instaloader.Post.from_shortcode(self.loader.context, shortcode)

            # Extracting information
        likes = post.likes
        comments = post.comments
        video_url = post.video_url if post.is_video else "N/A"

        print(f"Likes: {likes}")
        print(f"Comments: {comments}")
        print(f"Video URL: {video_url}")

            # Scrape additional data such as shares, saves, and music (if available)
        # return self.scrape_additional_data(post_url)

        # except Exception as e:
        #     print(f"Failed to extract post info: {e}")

    def scrape_additional_data(self, post_url):
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(post_url, headers=headers)

        # if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Look for JSON data in any script tag
        script_tags = soup.find_all('script', type="application/ld+json")
        for script in script_tags:
            # try:
            json_data = json.loads(script.string)
            print(json_data)
            if "interactionStatistic" in json_data:
                likes = json_data["interactionStatistic"][0].get("userInteractionCount", "N/A")
                comments = json_data["interactionStatistic"][1].get("userInteractionCount", "N/A")
                saves = json_data["interactionStatistic"][2].get("userInteractionCount", "N/A")

                print(f"Saves: {saves}")
                print(f"Shares: N/A")  # Placeholder
                print(f"Music: N/A")  # Placeholder
                break
            # except json.JSONDecodeError:
            #     continue

        # else:
            # print("Failed to scrape additional data.")

# Main script
if __name__ == "__main__":
    post_url = input("Enter the Instagram post URL: ")
    extractor = InstagramPostExtractor()
    # extractor.extract_post_info(post_url)
    extractor.scrape_additional_data(post_url)


Enter the Instagram post URL: https://www.instagram.com/p/C9mSaoxSO60/
Logged in successfully.
